# Task Description

Case description:
Welcome to East Carmen Airlines! As a strategic data scientist, you are confronted with many use cases to digitalize the airline business. Most important for airlines are accurate predictions of flight arrivals during a day of operations. When it comes to passenger handling, such predictions would admit a proactive steering, in order that the passengers reach the connection flights and therefore save a lot of money.

Your model should be applicable to each aircraft in East Carmen’s fleet. Let us take as an example the aircraft with registration name EC-LPD. At scheduled departure of the earliest morning flight from East Carmen’s hub Madrid the model should predict all flights of EC-LPD on the given day.

Many business stakeholders do not trust in the robustness of such a model. They claim that for a given aircraft, one could predict the first flight of the day with a decent error, but the errors of second and third flights will be huge. It is now your task to convince the business stakeholders that with a well-prepared data set and meaningful features you can deliver robust flight predictions.

Project Aim:
From different departments business analysts have collected a lot of information on possible features for flight predictions. Now it is your task to use these data to deliver a prototype forecasting model for all flights of a given aircraft at scheduled departure of the first flight of the day. 

For instance, for an aircraft with registration name EC-LPD, first flight of the day (FFOD) scheduled at 5:30 am from MAD to VIE and with the connection chain MAD-VIE-MAD-CDG-MAD-LHR-MAD your model should predict the successive landing times in VIE, MAD, CDG, MAD, LHR, MAD. The colleagues from daily operations are especially interested in how the confidence level of such a model evolves during a day of operations.

# General Information
| Amendments by me


3.1 File Names
- zip-folder: use_case_3.zip
- flight_information.csv : information on flights in a given time period, where each data point is a so-called 
leg, i.e. a flight from departure airport to destination airport with all relevant flight and crew information
- ground_information.csv : information on the ground processes after landing at our hub “East Carmen”,
where each data point describes the processes between two flights (inbound = landing, outbound = takeoff).


3.2 Column Description – Flight Information
- leg_no: unique identifier of a flight on a given day, at a certain time, with a given flight number
- fn_carrier: airline name
- fn_number: flight number; has to be unique on a given day
- dep_ap_sched: scheduled departure airport
- arr_ap_sched: scheduled arrival airport
- dep_sched_date: scheduled departure date
- dep_sched_time: scheduled departure time
- arr_sched_date: scheduled arrival date
- arr_sched_time: scheduled arrival time
- m_offblockdt: timestamp of departure
- m_onblockdt: timestamp of arrival
- ac_registration: aircraft registration number, i.e. the “license plate” of the aircraft
- change_reason_code: reason for delay (assigned after the flight)
- dep_delay: departure delay
- Ac Type Code: aircraft type (example: 320 = Airbus A320)
- trans_time: true minimal transition time for crew members after flight, i.e. transition time of the crew
member with the least time
- sched_trans_time: scheduled minimal transition time for crew members after flight, i.e. scheduled transition
time of the crew member with the least time
4
- Crew Group: assignment of what happens to the whole crew after a flight
o Start : First flight of day
o A : all crew members stay on the aircraft for the next flight
o B, B2: all crew members switch aircraft for the next flight
o C: at least one crew member switches aircraft for the next flight
- TLC_trans: names of crew members on flight with some additional information attached to each name (but business does not know exactly which additional information is visible there)
- crew_type_change: rank (cp = pilot, ca = cabin member) of crew members, who changed aircraft
- Sched Groundtime: scheduled ground time of the aircraft between flights | has mistakes; mostly (dep_sched_time of next line) - (arr_sched_time of current line); groundtime AFTER landing
- Act Groundtime: actual ground time of the aircraft between flights | has mistakes; mostly (m_offblocktime of next line) - (m_onblocktime of current line); groundtime AFTER landing


3.3 Column Description – Ground Information
- Each datapoint consists of an inbound (=arriving) flight and an outbound (=departing) flight from our hub “East Carmen)
- Catering, cleaning and pax (=passenger) boarding durations given for our hub “East Carmen” between the inbound and outbound flights
- day_of_origin: day of flight (also given in Flight Information)
- ac_type: aircraft type (also given in Flight Information)
- fn_number: flight number (also given in Flight Information)
- ac_registration: aircraft registration (also given in Flight Information)
- mingt: minimal scheduled ground time for the given aircraft
- dep_leg_inbound: departure airport name of inbound (=arriving) flight
- arr_leg_inbound: arrival airport name of inbound (=arriving) flight
- arr_leg_outbound: departure airport name of outbound (=departing) flight
- sched_inbound_dep: scheduled departure time of inbound (=arriving) flight
- sched_inbound_arr: scheduled arrival time of inbound (=arriving) flight
- sched_outbound_dep: scheduled departure time of outbound (=departing) flight
- sched_outbound_arr: scheduled arrival time of outbound (=departing) flight
- sched_turnaround: scheduled ground time for aircraft
- leg_inbound: leg number of inbound flight
- leg_outbound: leg number of outbound flight
- catering_duration: catering duration (i.e. filling up meal boxes) between flights in minutes
- cleaning_duration: cleaning duration between flights in minutes
- pax_boarding_duration: boarding duration between flights in minutes


3.4 Additional Info from Business Side:
- BI (=business intelligence) colleagues have collected a lot of flight information from many different de-partments: flight-, crew- and ground operations. Each department uses a different data warehouse. BI colleagues claim that some of the columns might be redundant and some information could be wrong, due to bad and not consistent data in the different data warehouses of the business departments. It is central for this prototype study to clean and transform the data and make consistency checks, before applying a machine learning algorithm.

In [25]:
# Imports
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

import datetime

import functions # own helper functions

#Settings
pd.set_option("display.max_rows", 70)
pd.set_option("display.max_columns", 101)


# Scope of this workbook

This workbook aims at initial feature creation and engineering without validating them. Validation is done in advanced eda with final preparation in advanced feature engineering. 

# 1. Full df import

In [26]:
# data frame for all edits
full_df = pd.read_pickle('../data/interim/full_df_unedited.pkl')

# Sorting data frame correctly
full_df.sort_values(by = ['ac_registration_x', 'm_offblockdt'], inplace = True)


# 4. Full df - Data Cleaning

This is the overall standard data frame we will use for all models except the baseline model. Here we do all the data cleaning we found out in raw_eda

## 4.1 Impute and Correct Values

In [27]:
# Use ac_type from ground dataset (which is more precise) for flight dataset AC Type
ac_subtypes = full_df[['ac_type','ac_registration_x']].drop_duplicates().dropna()

full_df = pd.merge(left = full_df, right = ac_subtypes, how = 'left', on = 'ac_registration_x')
full_df['Ac Type Code'] = full_df['ac_type_y']
full_df.drop(['ac_type_x', 'ac_type_y'], axis = 1, inplace = True)
full_df.tail()

,leg_no,fn_carrier,fn_number_x,dep_ap_sched,arr_ap_sched,dep_sched_date,dep_sched_time,arr_sched_date,arr_sched_time,m_offblockdt,m_onblockdt,ac_registration_x,change_reason_code,dep_delay,Ac Type Code,trans_time,sched_trans_time,Crew Group,TLC_trans,crew_type_change,Sched Groundtime,Act Groundtime,day_of_origin,fn_number_y,ac_registration_y,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,cleaning_duration,pax_boarding_duration
12096,272211046,East Carmen Airlines,EC3234,Lisamouth,East Carmen,2019-06-30,2019-06-30 10:30:00,2019-06-30,2019-06-30 12:10:00,2019-06-30 10:34:00,2019-06-30 12:05:00,ECLXEX,other problem,4.0,320,37,45,A,"['Tyler Welch_109_37_45_ca', 'Kenneth Alvarado...",[],50.0,85.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
12097,271969272,East Carmen Airlines,EC2687,East Carmen,West Jason,2019-06-30,2019-06-30 13:00:00,2019-06-30,2019-06-30 14:15:00,2019-06-30 13:30:00,2019-06-30 14:36:00,ECLXEX,rotational problem,30.0,320,0,0,Start,"['Carol Owens_nan_nan_nan_ca', 'Andrew Case_na...",[],45.0,54.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
12098,271969452,East Carmen Airlines,EC2688,West Jason,East Carmen,2019-06-30,2019-06-30 15:00:00,2019-06-30,2019-06-30 16:20:00,2019-06-30 15:30:00,2019-06-30 16:40:00,ECLXEX,rotational problem,30.0,320,41,45,A,"['Carol Owens_210_41_45_ca', 'Andrew Case_210_...",[],45.0,57.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
12099,272187492,East Carmen Airlines,EC2693,East Carmen,West Jason,2019-06-30,2019-06-30 17:05:00,2019-06-30,2019-06-30 18:20:00,2019-06-30 17:37:00,2019-06-30 19:01:00,ECLXEX,rotational problem,32.0,320,50,45,A,"['David Taylor_252_50_45_cp', 'Carol Owens_252...",[],45.0,4.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN
12100,272187671,East Carmen Airlines,EC2694,West Jason,East Carmen,2019-06-30,2019-06-30 19:05:00,2019-06-30,2019-06-30 20:20:00,2019-06-30 19:05:00,2019-06-30 20:15:00,ECLXEX,no reason,0.0,320,32,45,A,"['David Taylor_315_32_45_cp', 'Diana Smith_315...",[],NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN


## 4.1 Specific Add Columns 

In [28]:
# Create flt event number
#full_df["flt_event_number"] = full_df.groupby(['dep_sched_date', 'ac_registration_x']).cumcount()+1

functions.create_leg_no(full_df)

In [29]:
# Create leg as directional route 

functions.create_leg(full_df)

In [30]:
# Create rotation mismatch indicator
full_df = functions.create_rotmismatch(full_df)

idx = full_df[full_df['rot_mismatch']==True].index
idx_rol = []
for i in idx:
    idx_rol.append(i-1)
    idx_rol.append(i)
    idx_rol.append(i+1)

print(len(idx), " rotational mistakes")
full_df.loc[idx_rol,:].head(6)

222  rotational mistakes


,leg_no,fn_carrier,fn_number_x,dep_ap_sched,arr_ap_sched,dep_sched_date,dep_sched_time,arr_sched_date,arr_sched_time,m_offblockdt,m_onblockdt,ac_registration_x,change_reason_code,dep_delay,Ac Type Code,trans_time,sched_trans_time,Crew Group,TLC_trans,crew_type_change,Sched Groundtime,Act Groundtime,day_of_origin,fn_number_y,ac_registration_y,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,cleaning_duration,pax_boarding_duration,flt_event_number,leg,rot_mismatch
373,271957792,East Carmen Airlines,EC2581,East Carmen,Lake Lawrencechester,2019-06-12,2019-06-12 15:10:00,2019-06-12,2019-06-12 16:40:00,2019-06-12 16:29:00,2019-06-12 17:51:00,ECLBCX,technical problem,79.0,321,59,85,B,"['Scott Torres_209_59_85_cp', 'Ryan Brown_209_...",[],70.0,9.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,5,East Carmen-Lake Lawrencechester,False
374,271958002,East Carmen Airlines,EC2582,Lake Lawrencechester,East Carmen,2019-06-12,2019-06-12 17:50:00,2019-06-12,2019-06-12 19:15:00,2019-06-12 18:00:00,2019-06-12 19:19:00,ECLBCX,other problem,10.0,321,47,70,A,"['David Cruz_286_47_70_ca', 'Veronica Nunez_28...",[],NaN,NaN,2019-06-12,EC2582,ECLBCX,45.0,Lake Lawrencechester,East Carmen,East Latashaview,2019-06-12 17:50:00,2019-06-12 19:15:00,2019-06-12 18:15:00,2019-06-12 21:40:00,-60.0,271958002.0,272213781.0,23.0,NaN,19.0,6,Lake Lawrencechester-East Carmen,True
375,272027082,East Carmen Airlines,EC3316,East Latashaview,East Carmen,2019-06-13,2019-06-13 03:15:00,2019-06-13,2019-06-13 07:05:00,2019-06-13 03:34:00,2019-06-13 07:16:00,ECLBCX,other problem,19.0,321,0,0,Start,"['Reginald Grant_nan_nan_nan_cp', 'Jason Chane...",[],75.0,132.0,2019-06-13,EC3316,ECLBCX,45.0,East Latashaview,East Carmen,South Nathaniel,2019-06-13 03:15:00,2019-06-13 07:05:00,2019-06-13 08:20:00,2019-06-13 10:35:00,75.0,272027082.0,272022242.0,15.0,NaN,33.0,1,East Latashaview-East Carmen,False
549,272023984,East Carmen Airlines,EC3287,East Carmen,New Jessica,2019-06-20,2019-06-20 08:20:00,2019-06-20,2019-06-20 11:25:00,2019-06-20 08:38:00,2019-06-20 11:45:00,ECLBDX,other problem,18.0,322,68,60,B,"['Justin Medina_71_68_60_ca', 'Blake Rodriguez...",[],145.0,148.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2,East Carmen-New Jessica,False
550,272024194,East Carmen Airlines,EC3288,New Jessica,East Carmen,2019-06-20,2019-06-20 13:50:00,2019-06-20,2019-06-20 17:15:00,2019-06-20 14:13:00,2019-06-20 17:24:00,ECLBDX,other problem,23.0,322,0,0,Start,"['Angel Vazquez_nan_nan_nan_ca', 'Patrick Brow...",[],NaN,NaN,2019-06-20,EC3288,ECLBDX,45.0,New Jessica,East Carmen,East Latashaview,2019-06-20 13:50:00,2019-06-20 17:15:00,2019-06-20 18:15:00,2019-06-20 21:40:00,60.0,272024194.0,272213789.0,25.0,NaN,20.0,3,New Jessica-East Carmen,True
551,272027090,East Carmen Airlines,EC3316,East Latashaview,East Carmen,2019-06-21,2019-06-21 03:15:00,2019-06-21,2019-06-21 07:05:00,2019-06-21 03:33:00,2019-06-21 07:11:00,ECLBDX,other problem,18.0,322,0,0,Start,"['Emily Williams_nan_nan_nan_ca', 'Jerry Tyler...",[],75.0,90.0,2019-06-21,EC3316,ECLBDX,45.0,East Latashaview,East Carmen,New Jessica,2019-06-21 03:15:00,2019-06-21 07:05:00,2019-06-21 08:20:00,2019-06-21 11:25:00,75.0,272027090.0,272023985.0,29.0,NaN,22.0,1,East Latashaview-East Carmen,False


In [31]:
# Create delay columns
functions.create_actualblocktime(full_df)
functions.create_blockdelay(full_df)
functions.create_grounddelay(full_df)

In [32]:
# Create new features for day of the week and hour of the day
full_df['day_of_week'] = full_df['dep_sched_time'].dt.dayofweek
full_df['hour_of_day_dep'] = full_df['dep_sched_time'].dt.hour
full_df['hour_of_day_arr'] = full_df['arr_sched_time'].dt.hour
full_df['minute_of_day_dep'] = full_df['dep_sched_time'].dt.hour*60+full_df['dep_sched_time'].dt.minute

In [33]:
# Create correct crew change column

functions.create_crewchange_after(full_df)

full_df['Crewchange'].value_counts()

no change          5207
both               4127
last flt of day    2013
cabin               453
cockpit             301
Name: Crewchange, dtype: int64

In [34]:
# Correcting the ground times to display the ground values before a flight, not after
functions.create_groundtimes_before(full_df)


In [35]:

# Create crew change column before flight
functions.create_crewchange_before(full_df)
full_df['Crewchange_before'].value_counts()

full_df.head(5)

,leg_no,fn_carrier,fn_number_x,dep_ap_sched,arr_ap_sched,dep_sched_date,dep_sched_time,arr_sched_date,arr_sched_time,m_offblockdt,m_onblockdt,ac_registration_x,change_reason_code,dep_delay,Ac Type Code,trans_time,sched_trans_time,Crew Group,TLC_trans,crew_type_change,Sched Groundtime,Act Groundtime,day_of_origin,fn_number_y,ac_registration_y,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,cleaning_duration,pax_boarding_duration,flt_event_number,leg,rot_mismatch,actual_block_time,block_delay,ground_delay,day_of_week,hour_of_day_dep,hour_of_day_arr,minute_of_day_dep,Crewchange,act_groundtime_before,sched_groundtime_before,Crewchange_before
0,272024970,East Carmen Airlines,EC3292,New Jessica,East Carmen,2019-06-01,2019-06-01 03:25:00,2019-06-01,2019-06-01 06:45:00,2019-06-01 03:50:00,2019-06-01 07:01:00,ECLBAX,other problem,25.0,321,0,0,Start,"['Sean Weeks_nan_nan_nan_ca', 'Tony Lloyd_nan_...",[],95.0,94.0,2019-06-01,EC3292,ECLBAX,45.0,New Jessica,East Carmen,South Nathaniel,2019-06-01 03:25:00,2019-06-01 06:45:00,2019-06-01 08:20:00,2019-06-01 10:35:00,95.0,272024970.0,272022230.0,27.0,NaN,25.0,1,New Jessica-East Carmen,False,191.0,-9.0,25.0,5,3,6,205,both,94.0,95.0,first flt of day
1,272022230,East Carmen Airlines,EC3257,East Carmen,South Nathaniel,2019-06-01,2019-06-01 08:20:00,2019-06-01,2019-06-01 10:35:00,2019-06-01 08:35:00,2019-06-01 10:41:00,ECLBAX,other problem,15.0,321,60,60,B,"['Toni Rodriguez_48_60_60_ca', 'Rachel Smith_4...",[],75.0,120.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2,East Carmen-South Nathaniel,False,126.0,-9.0,15.0,5,8,10,500,no change,94.0,95.0,both
2,272212848,East Carmen Airlines,EC3258,South Nathaniel,East Carmen,2019-06-01,2019-06-01 11:50:00,2019-06-01,2019-06-01 14:15:00,2019-06-01 12:41:00,2019-06-01 14:52:00,ECLBAX,rotational problem,51.0,321,68,75,A,"['Toni Rodriguez_127_68_75_ca', 'Rachel Smith_...",[],80.0,86.0,2019-06-01,EC3258,ECLBAX,45.0,South Nathaniel,East Carmen,Joneshaven,2019-06-01 11:50:00,2019-06-01 14:15:00,2019-06-01 15:35:00,2019-06-01 17:00:00,80.0,272212848.0,271997824.0,25.0,NaN,NaN,3,South Nathaniel-East Carmen,False,131.0,-14.0,51.0,5,11,14,710,both,120.0,75.0,no change
3,271997824,East Carmen Airlines,EC3021,East Carmen,Joneshaven,2019-06-01,2019-06-01 15:35:00,2019-06-01,2019-06-01 17:00:00,2019-06-01 16:18:00,2019-06-01 17:32:00,ECLBAX,rotational problem,43.0,321,64,70,B2,"['Tyler Morris_224_64_70_ca', 'Sergio Cummings...","['ca', 'cp', 'ca', 'ca', 'ca']",50.0,32.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,East Carmen-Joneshaven,False,74.0,-11.0,43.0,5,15,17,935,no change,86.0,80.0,both
4,271998033,East Carmen Airlines,EC3022,Joneshaven,East Carmen,2019-06-01,2019-06-01 17:50:00,2019-06-01,2019-06-01 19:10:00,2019-06-01 18:04:00,2019-06-01 19:14:00,ECLBAX,other problem,14.0,321,42,50,A,"['Sergio Cummings_279_42_50_cp', 'Jackie Black...",[],NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,5,Joneshaven-East Carmen,False,70.0,-10.0,14.0,5,17,19,1070,last flt of day,32.0,50.0,no change


## 4.2 Specific Drop Rows 

In [36]:
# Mark dupes to drop 
print(full_df.shape)
full_df.drop_duplicates(subset = ['leg_no'], inplace = True, keep = False) # we keep no dupes at all
print(full_df.shape)

(12101, 53)
(11937, 53)


In [37]:
# Drop NAs for ground delay, block delay
print(full_df.shape)
full_df.dropna(subset=['block_delay'], inplace = True)
print(full_df.shape)
full_df.dropna(subset=['ground_delay'], inplace = True)
print(full_df.shape)

(11937, 53)
(11858, 53)
(11858, 53)


In [38]:
# Find cases where dep ap flight is not equal dep ap ground
print(full_df[full_df['dep_leg_inbound'].notna()][full_df['dep_ap_sched']!=full_df['dep_leg_inbound']].shape)
print(full_df[full_df['arr_leg_inbound'].notna()][full_df['arr_ap_sched']!=full_df['arr_leg_inbound']].shape)

(0, 53)
(0, 53)


C:\Users\PC Paul\AppData\Local\Temp\ipykernel_10780\2900044449.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(full_df[full_df['dep_leg_inbound'].notna()][full_df['dep_ap_sched']!=full_df['dep_leg_inbound']].shape)
C:\Users\PC Paul\AppData\Local\Temp\ipykernel_10780\2900044449.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  print(full_df[full_df['arr_leg_inbound'].notna()][full_df['arr_ap_sched']!=full_df['arr_leg_inbound']].shape)


In [39]:
# Mark rows that have wrong sched/act ground time 

def recalc_groundtimes_no_replace(df):
    df['act_groundtime_new'] = np.where(
        (df['ac_registration_x'].shift(-1).eq(df['ac_registration_x']) == True) & (df['Act Groundtime'].isna() == False),
        np.around((df['m_offblockdt'].shift(-1) - df['m_onblockdt']).dt.total_seconds()/60, decimals =1),
        df['Act Groundtime']
    )
    df['sched_groundtime_new'] = np.where(
        (df['ac_registration_x'].shift(-1).eq(df['ac_registration_x']) == True) & (df['Sched Groundtime'].isna() == False),
        np.around((df['dep_sched_time'].shift(-1) - df['arr_sched_time']).dt.total_seconds()/60, decimals =1),
        df['Sched Groundtime']
    )

recalc_groundtimes_no_replace(full_df)


# Mark rows which do not have correct actual or sched ground time

full_df['act_gt_diff'] = np.around(full_df['Act Groundtime'] - full_df['act_groundtime_new'],0)
full_df['sched_gt_diff'] = np.around(full_df['Sched Groundtime'] - full_df['sched_groundtime_new'],0)
#print(str(full_df[full_df['act_groundtime_new'].isna() == False].shape[0]) + " rows have wrongly calculated ground times. We filter out NAs for Act Groundtime as the majority of them" +
#    " are the last flights of the respective day.")

# Fill NAs which are created as some ground times are NA
full_df['act_gt_diff'].fillna(0, inplace = True)
full_df['sched_gt_diff'].fillna(0, inplace = True)

full_df["rows_to_drop"] = np.where(full_df['act_gt_diff']!=0, 1, 0)
print(full_df["rows_to_drop"].sum())
full_df["rows_to_drop"] = np.where(full_df['sched_gt_diff']!=0, 1, full_df["rows_to_drop"] )
print(full_df["rows_to_drop"].sum())

# Drop unneeded columns again 
full_df.drop(['act_gt_diff', 'sched_gt_diff', 'act_groundtime_new', 'sched_groundtime_new'], inplace = True, axis = 1)
full_df.shape

142
143


(11858, 54)

In [40]:
# Mark rows with rotational mistakes
full_df["rows_to_drop"] = np.where(full_df['rot_mismatch']==True, 1, full_df["rows_to_drop"])
full_df["rows_to_drop"].sum()

201

In [41]:

# Create Feature for additional rows to drop for ground delay prediction
full_df['rows_to_drop_grounddelay'] = np.where(full_df['Sched Groundtime'].isna() == True, 1, 0)
full_df["rows_to_drop_grounddelay"].sum()

1992

In [42]:

# Also drop all last flights of a day for grounddelay
full_df['rows_to_drop_grounddelay'] = np.where(full_df['Crewchange']=='last flt of day', 1, full_df['rows_to_drop_grounddelay'])
full_df["rows_to_drop_grounddelay"].sum()

1997

In [43]:
# Actual drop of rows

full_df = full_df[full_df['rows_to_drop'] == 0]
full_df.shape

(11657, 55)

In [44]:
full_df.head()

,leg_no,fn_carrier,fn_number_x,dep_ap_sched,arr_ap_sched,dep_sched_date,dep_sched_time,arr_sched_date,arr_sched_time,m_offblockdt,m_onblockdt,ac_registration_x,change_reason_code,dep_delay,Ac Type Code,trans_time,sched_trans_time,Crew Group,TLC_trans,crew_type_change,Sched Groundtime,Act Groundtime,day_of_origin,fn_number_y,ac_registration_y,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,cleaning_duration,pax_boarding_duration,flt_event_number,leg,rot_mismatch,actual_block_time,block_delay,ground_delay,day_of_week,hour_of_day_dep,hour_of_day_arr,minute_of_day_dep,Crewchange,act_groundtime_before,sched_groundtime_before,Crewchange_before,rows_to_drop,rows_to_drop_grounddelay
0,272024970,East Carmen Airlines,EC3292,New Jessica,East Carmen,2019-06-01,2019-06-01 03:25:00,2019-06-01,2019-06-01 06:45:00,2019-06-01 03:50:00,2019-06-01 07:01:00,ECLBAX,other problem,25.0,321,0,0,Start,"['Sean Weeks_nan_nan_nan_ca', 'Tony Lloyd_nan_...",[],95.0,94.0,2019-06-01,EC3292,ECLBAX,45.0,New Jessica,East Carmen,South Nathaniel,2019-06-01 03:25:00,2019-06-01 06:45:00,2019-06-01 08:20:00,2019-06-01 10:35:00,95.0,272024970.0,272022230.0,27.0,NaN,25.0,1,New Jessica-East Carmen,False,191.0,-9.0,25.0,5,3,6,205,both,94.0,95.0,first flt of day,0,0
1,272022230,East Carmen Airlines,EC3257,East Carmen,South Nathaniel,2019-06-01,2019-06-01 08:20:00,2019-06-01,2019-06-01 10:35:00,2019-06-01 08:35:00,2019-06-01 10:41:00,ECLBAX,other problem,15.0,321,60,60,B,"['Toni Rodriguez_48_60_60_ca', 'Rachel Smith_4...",[],75.0,120.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2,East Carmen-South Nathaniel,False,126.0,-9.0,15.0,5,8,10,500,no change,94.0,95.0,both,0,0
2,272212848,East Carmen Airlines,EC3258,South Nathaniel,East Carmen,2019-06-01,2019-06-01 11:50:00,2019-06-01,2019-06-01 14:15:00,2019-06-01 12:41:00,2019-06-01 14:52:00,ECLBAX,rotational problem,51.0,321,68,75,A,"['Toni Rodriguez_127_68_75_ca', 'Rachel Smith_...",[],80.0,86.0,2019-06-01,EC3258,ECLBAX,45.0,South Nathaniel,East Carmen,Joneshaven,2019-06-01 11:50:00,2019-06-01 14:15:00,2019-06-01 15:35:00,2019-06-01 17:00:00,80.0,272212848.0,271997824.0,25.0,NaN,NaN,3,South Nathaniel-East Carmen,False,131.0,-14.0,51.0,5,11,14,710,both,120.0,75.0,no change,0,0
3,271997824,East Carmen Airlines,EC3021,East Carmen,Joneshaven,2019-06-01,2019-06-01 15:35:00,2019-06-01,2019-06-01 17:00:00,2019-06-01 16:18:00,2019-06-01 17:32:00,ECLBAX,rotational problem,43.0,321,64,70,B2,"['Tyler Morris_224_64_70_ca', 'Sergio Cummings...","['ca', 'cp', 'ca', 'ca', 'ca']",50.0,32.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,East Carmen-Joneshaven,False,74.0,-11.0,43.0,5,15,17,935,no change,86.0,80.0,both,0,0
4,271998033,East Carmen Airlines,EC3022,Joneshaven,East Carmen,2019-06-01,2019-06-01 17:50:00,2019-06-01,2019-06-01 19:10:00,2019-06-01 18:04:00,2019-06-01 19:14:00,ECLBAX,other problem,14.0,321,42,50,A,"['Sergio Cummings_279_42_50_cp', 'Jackie Black...",[],NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,5,Joneshaven-East Carmen,False,70.0,-10.0,14.0,5,17,19,1070,last flt of day,32.0,50.0,no change,0,1


## 4.3 Specific Drop Columns 

In [45]:
full_df.head()

,leg_no,fn_carrier,fn_number_x,dep_ap_sched,arr_ap_sched,dep_sched_date,dep_sched_time,arr_sched_date,arr_sched_time,m_offblockdt,m_onblockdt,ac_registration_x,change_reason_code,dep_delay,Ac Type Code,trans_time,sched_trans_time,Crew Group,TLC_trans,crew_type_change,Sched Groundtime,Act Groundtime,day_of_origin,fn_number_y,ac_registration_y,mingt,dep_leg_inbound,arr_leg_inbound,arr_leg_outbound,sched_inbound_dep,sched_inbound_arr,sched_outbound_dep,sched_outbound_arr,sched_turnaround,leg_inbound,leg_outbound,catering_duration,cleaning_duration,pax_boarding_duration,flt_event_number,leg,rot_mismatch,actual_block_time,block_delay,ground_delay,day_of_week,hour_of_day_dep,hour_of_day_arr,minute_of_day_dep,Crewchange,act_groundtime_before,sched_groundtime_before,Crewchange_before,rows_to_drop,rows_to_drop_grounddelay
0,272024970,East Carmen Airlines,EC3292,New Jessica,East Carmen,2019-06-01,2019-06-01 03:25:00,2019-06-01,2019-06-01 06:45:00,2019-06-01 03:50:00,2019-06-01 07:01:00,ECLBAX,other problem,25.0,321,0,0,Start,"['Sean Weeks_nan_nan_nan_ca', 'Tony Lloyd_nan_...",[],95.0,94.0,2019-06-01,EC3292,ECLBAX,45.0,New Jessica,East Carmen,South Nathaniel,2019-06-01 03:25:00,2019-06-01 06:45:00,2019-06-01 08:20:00,2019-06-01 10:35:00,95.0,272024970.0,272022230.0,27.0,NaN,25.0,1,New Jessica-East Carmen,False,191.0,-9.0,25.0,5,3,6,205,both,94.0,95.0,first flt of day,0,0
1,272022230,East Carmen Airlines,EC3257,East Carmen,South Nathaniel,2019-06-01,2019-06-01 08:20:00,2019-06-01,2019-06-01 10:35:00,2019-06-01 08:35:00,2019-06-01 10:41:00,ECLBAX,other problem,15.0,321,60,60,B,"['Toni Rodriguez_48_60_60_ca', 'Rachel Smith_4...",[],75.0,120.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,2,East Carmen-South Nathaniel,False,126.0,-9.0,15.0,5,8,10,500,no change,94.0,95.0,both,0,0
2,272212848,East Carmen Airlines,EC3258,South Nathaniel,East Carmen,2019-06-01,2019-06-01 11:50:00,2019-06-01,2019-06-01 14:15:00,2019-06-01 12:41:00,2019-06-01 14:52:00,ECLBAX,rotational problem,51.0,321,68,75,A,"['Toni Rodriguez_127_68_75_ca', 'Rachel Smith_...",[],80.0,86.0,2019-06-01,EC3258,ECLBAX,45.0,South Nathaniel,East Carmen,Joneshaven,2019-06-01 11:50:00,2019-06-01 14:15:00,2019-06-01 15:35:00,2019-06-01 17:00:00,80.0,272212848.0,271997824.0,25.0,NaN,NaN,3,South Nathaniel-East Carmen,False,131.0,-14.0,51.0,5,11,14,710,both,120.0,75.0,no change,0,0
3,271997824,East Carmen Airlines,EC3021,East Carmen,Joneshaven,2019-06-01,2019-06-01 15:35:00,2019-06-01,2019-06-01 17:00:00,2019-06-01 16:18:00,2019-06-01 17:32:00,ECLBAX,rotational problem,43.0,321,64,70,B2,"['Tyler Morris_224_64_70_ca', 'Sergio Cummings...","['ca', 'cp', 'ca', 'ca', 'ca']",50.0,32.0,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,4,East Carmen-Joneshaven,False,74.0,-11.0,43.0,5,15,17,935,no change,86.0,80.0,both,0,0
4,271998033,East Carmen Airlines,EC3022,Joneshaven,East Carmen,2019-06-01,2019-06-01 17:50:00,2019-06-01,2019-06-01 19:10:00,2019-06-01 18:04:00,2019-06-01 19:14:00,ECLBAX,other problem,14.0,321,42,50,A,"['Sergio Cummings_279_42_50_cp', 'Jackie Black...",[],NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,5,Joneshaven-East Carmen,False,70.0,-10.0,14.0,5,17,19,1070,last flt of day,32.0,50.0,no change,0,1


In [46]:
adv_drop_columns = [
    'Crew Group',
    'TLC_trans',
    'crew_type_change',
    'fn_number_x', # unreliable
    'dep_ap_sched', # encoded in route/leg; no additional information
    'arr_ap_sched', # encoded in route/leg; no additional information
    'Sched Groundtime', # As we have sched_groundtime_before and calculate landing times, Sched Groundtime does not have any predictive value for us
    'Act Groundtime' # same
]

In [47]:
full_df = full_df.drop(adv_drop_columns, axis = 1)
full_df.shape

(11657, 47)

## 4.4 Save full_df

In [48]:
full_df.to_pickle('../data/processed/full_df.pkl')
full_df.to_csv('../data/processed/full_df.csv')

In [50]:
# temp

ac_registration_x,ECLBAX,ECLBBX,ECLBCX,ECLBDX,ECLBEX,ECLBFX,ECLBIX,ECLBJX,ECLBKX,ECLBLX,ECLBMX,ECLBNX,ECLBOX,ECLBPX,ECLBQX,ECLBRX,ECLBSX,ECLBTX,ECLBUX,ECLBVX,ECLBWX,ECLBXX,ECLBYX,ECLBZX,ECLDAX,ECLDBX,ECLDCX,ECLDDX,ECLDEX,ECLDFX,ECLDGX,ECLGAX,ECLGBX,ECLGCX,ECLGFX,ECLGGX,ECLGHX,ECLGIX,ECLGJX,ECLGKX,ECLGLX,ECLGMX,ECLGNX,ECLGOX,ECLGPX,ECLGQX,ECLGRX,ECLWAX,ECLWBX,ECLWCX,ECLWDX,ECLWEX,ECLWFX,ECLWGX,ECLWHX,ECLWIX,ECLWJX,ECLWKX,ECLWLX,ECLWMX,ECLWNX,ECLWOX,ECLWPX,ECLWQX,ECLXAX,ECLXBX,ECLXCX,ECLXDX,ECLXEX
mingt,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
30.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,20,16,10,19,8,19,18,19,9,25,20,4,12,13,17,16,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
35.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,53,32,28,35,31,33,50,30,20,40,27,8,37,38,25,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
40.0,0,0,0,0,0,0,64,53,60,61,64,61,49,72,67,64,59,59,61,67,69,64,72,65,61,49,70,58,61,71,60,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,69,75,64,66,64,62,78,59,55,57,63,57,77,62,49,49,75,64,78,69,57,69
45.0,54,55,55,50,58,50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
